# Libraries

In [1]:
# Importing libraries
import pandas as pd
import numpy as np

# Option Setting

In [2]:
# Setting options for readability
pd.set_option("max_colwidth", None)
pd.set_option("display.max_columns", None)

# Loading Dataframes

In [3]:
# Loading data dictionary to understand the column definitions
data_dictionary = pd.read_csv('data_dictionary.csv')

# Viewing data_dictionary
data_dictionary

,Field,Description
0,VendorID,"A code indicating the LPEP provider that provided the record (1= Creative Mobile Technologies, LLC; 2= Verifone Inc.)"
1,lpep_pickup_datetime,The date and time when the meter was engaged
2,lpep_dropoff_datetime,The date and time when the meter was disengaged
3,store_and_fwd_flag,"This flag indicates whether the trip record was held in vehicle memory before sending to the vendor, aka �store and forward,� because the vehicle did not have a connection to the server (Y= store and forward trip; N= not a store and forward trip)"
4,RatecodeID,The final rate code in effect at the end of the trip (1= Standard rate; 2= JFK; 3= Newark; 4= Nassau or Westchester; 5= Negotiated fare; 6= Group ride)
5,PULocationID,TLC Taxi Zone in which the taximeter was engaged
6,DOLocationID,TLC Taxi Zone in which the taximeter was disengaged
7,passenger_count,The number of passengers in the vehicle (this is a driver entered value)
8,trip_distance,The elapsed trip distance in miles reported by the taximeter
9,fare_amount,The time-and-distance fare calculated by the meter


In [4]:
# Loading taxi zones to check what's included
taxi_zones = pd.read_csv('taxi_zones.csv')

# Viewing taxi_zones
taxi_zones

,LocationID,Borough,Zone,service_zone
0,1,EWR,Newark Airport,EWR
1,2,Queens,Jamaica Bay,Boro Zone
2,3,Bronx,Allerton/Pelham Gardens,Boro Zone
3,4,Manhattan,Alphabet City,Yellow Zone
4,5,Staten Island,Arden Heights,Boro Zone
...,...,...,...,...
260,261,Manhattan,World Trade Center,Yellow Zone
261,262,Manhattan,Yorkville East,Yellow Zone
262,263,Manhattan,Yorkville West,Yellow Zone
263,264,Unknown,NV,NaN


taxi_zones contains location details. This will be useful when we want to expand the main dataframe to include location details.

In [5]:
# Checking if there are any duplicate rows before merging. No duplicate rows if False
taxi_zones['LocationID'].duplicated().any()

False

In [6]:
# Checking info
taxi_zones.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 265 entries, 0 to 264
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   LocationID    265 non-null    int64 
 1   Borough       265 non-null    object
 2   Zone          264 non-null    object
 3   service_zone  263 non-null    object
dtypes: int64(1), object(3)
memory usage: 8.4+ KB


There are two null values in service_zone and one null value in Zone which is visible at the bottom end of the taxi_zones dataframe.

In [7]:
# Filling null values with Unknown
taxi_zones.fillna('Unknown', inplace = True)

In [8]:
# Loading taxi trips for all four years
taxi_trips_2017 = pd.read_csv('taxi_trips/2017_taxi_trips.csv')
taxi_trips_2018 = pd.read_csv('taxi_trips/2018_taxi_trips.csv')
taxi_trips_2019 = pd.read_csv('taxi_trips/2019_taxi_trips.csv')
taxi_trips_2020 = pd.read_csv('taxi_trips/2020_taxi_trips.csv')

# Printing the shape of each dataframe
print(taxi_trips_2017.shape)
print(taxi_trips_2018.shape)
print(taxi_trips_2019.shape)
print(taxi_trips_2020.shape)

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


(11740667, 18)
(8807303, 18)
(6044050, 19)
(1734051, 19)


In [9]:
# Checking the info of dataframes
print(taxi_trips_2017.info())
print(taxi_trips_2018.info())
print(taxi_trips_2019.info())
print(taxi_trips_2020.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11740667 entries, 0 to 11740666
Data columns (total 18 columns):
 #   Column                 Dtype  
---  ------                 -----  
 0   VendorID               int64  
 1   lpep_pickup_datetime   object 
 2   lpep_dropoff_datetime  object 
 3   store_and_fwd_flag     object 
 4   RatecodeID             int64  
 5   PULocationID           int64  
 6   DOLocationID           int64  
 7   passenger_count        int64  
 8   trip_distance          float64
 9   fare_amount            float64
 10  extra                  float64
 11  mta_tax                float64
 12  tip_amount             float64
 13  tolls_amount           float64
 14  improvement_surcharge  float64
 15  total_amount           float64
 16  payment_type           int64  
 17  trip_type              float64
dtypes: float64(9), int64(6), object(3)
memory usage: 1.6+ GB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8807303 entries, 0 to 8807302
Data columns (tota

congestion_surcharge is missing from taxi_trips_2017 and taxi_trips_2018.

In [10]:
# Adding the column to make all dataframes same width before concatenating the dataframes
taxi_trips_2017['congestion_surcharge'] = 0.0
taxi_trips_2018['congestion_surcharge'] = 0.0

# Checking the shape to confirm we have 19 columns
print(taxi_trips_2017.shape)
print(taxi_trips_2018.shape)

(11740667, 19)
(8807303, 19)


In [11]:
# Concating all taxi trips
taxi_trips = pd.concat([taxi_trips_2017, taxi_trips_2018, taxi_trips_2019, taxi_trips_2020]).reset_index(drop = True)

# Checking the shape
taxi_trips.shape

(28326071, 19)

In [12]:
# Verifying the number of rows are as expected
print("Total rows in concatenated dataframe: {}".format(taxi_trips.shape[0]))
print("Rows in each dataframe added up: {}".format(taxi_trips_2017.shape[0] + taxi_trips_2018.shape[0] + taxi_trips_2019.shape[0] + taxi_trips_2020.shape[0]))

Total rows in concatenated dataframe: 28326071
Rows in each dataframe added up: 28326071


# Data Types

In [13]:
# Checking the info of our concatenated dataframe
taxi_trips.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28326071 entries, 0 to 28326070
Data columns (total 19 columns):
 #   Column                 Dtype  
---  ------                 -----  
 0   VendorID               float64
 1   lpep_pickup_datetime   object 
 2   lpep_dropoff_datetime  object 
 3   store_and_fwd_flag     object 
 4   RatecodeID             float64
 5   PULocationID           int64  
 6   DOLocationID           int64  
 7   passenger_count        float64
 8   trip_distance          float64
 9   fare_amount            float64
 10  extra                  float64
 11  mta_tax                float64
 12  tip_amount             float64
 13  tolls_amount           float64
 14  improvement_surcharge  float64
 15  total_amount           float64
 16  payment_type           float64
 17  trip_type              float64
 18  congestion_surcharge   float64
dtypes: float64(14), int64(2), object(3)
memory usage: 4.0+ GB


<div class="alert alert-block alert-info">
    <h4>Data Types Inferred from Data Dictionary</h4><br>
    <b>VendorID:</b> Integer<br>
    <b>lpep_pickup_datetime:</b> DateTime<br>
    <b>lpep_dropoff_datetime:</b> DateTime<br>
    <b>store_and_fwd_flag:</b> String<br>
    <b>RatecodeID:</b> Integer<br>
    <b>PULocationID:</b> Integer<br>
    <b>DOLocationID:</b> Integer<br>
    <b>passenger_count:</b> Integer<br>
    <b>trip_distance:</b> Float<br>
    <b>fare_amount:</b> Float<br>
    <b>extra:</b> Float<br>
    <b>mta_tax:</b> Float<br>
    <b>tip_amount:</b> Float<br>
    <b>tolls_amount:</b> Float<br>
    <b>improvement_surcharge:</b> Float<br>
    <b>total_amount:</b> Float<br>
    <b>payment_type:</b> Integer<br>
    <b>trip_type:</b> Integer<br>
    <b>congestion_surcharge:</b> Float<br>
</div>

###### Converting String to DateTime format

In [14]:
taxi_trips['lpep_pickup_datetime'] = pd.to_datetime(taxi_trips['lpep_pickup_datetime'])
taxi_trips['lpep_dropoff_datetime'] = pd.to_datetime(taxi_trips['lpep_dropoff_datetime'])

print(taxi_trips['lpep_pickup_datetime'].dtype)
print(taxi_trips['lpep_dropoff_datetime'].dtype)

datetime64[ns]
datetime64[ns]


###### Converting Float to Integer

In [15]:
# Checking values in columns that are float to see if the values are actually integers
print(taxi_trips['VendorID'].value_counts())
print(taxi_trips['RatecodeID'].value_counts())
print(taxi_trips['passenger_count'].value_counts())
print(taxi_trips['payment_type'].value_counts())
print(taxi_trips['trip_type'].value_counts())

2.0    22569744
1.0     4814128
Name: VendorID, dtype: int64
1.0     26555346
5.0       740222
2.0        55534
4.0        18498
3.0        13769
6.0          349
99.0         154
Name: RatecodeID, dtype: int64
1.0    23244224
2.0     2117741
5.0      909801
6.0      468953
3.0      455954
4.0      156794
0.0       29101
8.0         596
7.0         538
9.0         170
Name: passenger_count, dtype: int64
1.0    14793538
2.0    12392982
3.0      132220
4.0       63966
5.0        1166
Name: payment_type, dtype: int64
1.0    26640078
2.0      743340
Name: trip_type, dtype: int64


In [16]:
# Converting to integer
taxi_trips['VendorID'] = taxi_trips['VendorID'].fillna(0.0).astype(int)
taxi_trips['RatecodeID'] = taxi_trips['RatecodeID'].fillna(0.0).astype(int)
taxi_trips['passenger_count'] = taxi_trips['passenger_count'].fillna(0.0).astype(int)
taxi_trips['payment_type'] = taxi_trips['payment_type'].fillna(0.0).astype(int)
taxi_trips['trip_type'] = taxi_trips['trip_type'].fillna(0.0).astype(int)

# Mapping and Exploring the columns

In this section, we are going to explore all the columns to see what needs further investigation whilst mapping and merging columns.

In [17]:
# Checking for null values in the dataframe
taxi_trips.isnull().sum()

VendorID                       0
lpep_pickup_datetime           0
lpep_dropoff_datetime          0
store_and_fwd_flag        942199
RatecodeID                     0
PULocationID                   0
DOLocationID                   0
passenger_count                0
trip_distance                  0
fare_amount                    0
extra                          0
mta_tax                        0
tip_amount                     0
tolls_amount                   0
improvement_surcharge          2
total_amount                   0
payment_type                   0
trip_type                      0
congestion_surcharge     1488581
dtype: int64

### VendorID

A code indicating the LPEP provider that provided the record (1= Creative Mobile Technologies, LLC; 2= Verifone Inc.)

In [18]:
# Checking the value counts
taxi_trips['VendorID'].value_counts()

2    22569744
1     4814128
0      942199
Name: VendorID, dtype: int64

In [19]:
# Mapping the VendorID to the vendor
taxi_trips['Vendor'] = taxi_trips['VendorID'].map({1 : 'Creative Mobile Technologies, LLC', 2 : 'Verifone Inc.', 0 : 'Unknown'})

taxi_trips['Vendor'].value_counts()

Verifone Inc.                        22569744
Creative Mobile Technologies, LLC     4814128
Unknown                                942199
Name: Vendor, dtype: int64

There are 942199 records where the VendorID is not provided. We will look at the possibility of populating the names of unknowns vendor in the next section.

### lpep_pickup_datetime

The date and time when the meter was engaged

### lpep_dropoff_datetime

The date and time when the meter was disengaged

### store_and_fwd_flag

This flag indicates whether the trip record was held in vehicle memory before sending to the vendor, aka “store and forward,” because the vehicle did not have a connection to the server (Y= store and forward trip; N= not a store and forward trip)

In [20]:
# Checking the value counts
taxi_trips['store_and_fwd_flag'].value_counts()

N    27330950
Y       52922
Name: store_and_fwd_flag, dtype: int64

In [21]:
# Checking values in store_and_fwd_flag
print(taxi_trips['store_and_fwd_flag'].unique())

['N' 'Y' nan]


In [22]:
# How many null values are there in store_and_fwd_flag?
print("There are {} null values in store_and_fwd_flag".format(taxi_trips['store_and_fwd_flag'].isnull().sum()))

There are 942199 null values in store_and_fwd_flag


We will look at the possibility of populating the null values in store_and_fwd_flag in the next section.

### RatecodeID

The final rate code in effect at the end of the trip (1= Standard rate; 2= JFK; 3= Newark; 4= Nassau or Westchester; 5= Negotiated fare; 6= Group ride)

In [23]:
# Checking the value counts
taxi_trips['RatecodeID'].value_counts()

1     26555346
0       942199
5       740222
2        55534
4        18498
3        13769
6          349
99         154
Name: RatecodeID, dtype: int64

In [24]:
# Checking values in VendorID
print(taxi_trips['RatecodeID'].unique())

[ 1  2  4  3  6 99  5  0]


In [25]:
# Dictionary of ID and name
rate_codes = {1 : 'Standard rate', 2 : 'JFK', 3 : 'Newark', 4 : 'Nassau or Westchester', 5 : 'Negotiated fare', 6 : 'Group ride', 0 : 'Unknown', 99 : 'Unknown'}

# Mapping the RatecodeID to the rate code
taxi_trips['Ratecode'] = taxi_trips['RatecodeID'].map(rate_codes)

# Checking values in Ratecode
taxi_trips['Ratecode'].value_counts()

Standard rate            26555346
Unknown                    942353
Negotiated fare            740222
JFK                         55534
Nassau or Westchester       18498
Newark                      13769
Group ride                    349
Name: Ratecode, dtype: int64

In [26]:
# How many null values are there in Ratecode?
print("There are {} unknowns in Ratecode".format(taxi_trips.loc[taxi_trips['Ratecode']=='Unknown'].shape[0]))

There are 942353 unknowns in Ratecode


We will look at the possibility of populating the unknowns in Ratecode in the next section.

### PULocationID

TLC Taxi Zone in which the taximeter was engaged

In [27]:
# Checking the value counts
taxi_trips['PULocationID'].value_counts()

74     1894141
75     1629516
41     1587500
7      1300642
82     1208719
        ...   
204         40
12          33
99          19
199         15
105          1
Name: PULocationID, Length: 262, dtype: int64

In [28]:
# How many null values are there in PULocationID?
print("There are {} null values in PULocationID".format(taxi_trips['PULocationID'].isnull().sum()))

There are 0 null values in PULocationID


In [29]:
# Checking the lowest and highest numbers in PULocationID
print(taxi_trips['PULocationID'].min())
print(taxi_trips['PULocationID'].max())

1
265


In [30]:
# Merge PULocationID with taxi_zones
taxi_trips = taxi_trips.merge(right=taxi_zones, how='left', left_on='PULocationID', right_on='LocationID')

In [31]:
# Dropping LocationID from taxi_trips that was merged from taxi_zones
taxi_trips.drop(columns='LocationID', inplace=True)

In [32]:
# Renaming merged columns
taxi_trips.rename(columns={'Borough' : 'PU_Borough', 'Zone' : 'PU_Zone', 'service_zone' : 'PU_service_zone'}, inplace=True)

### DOLocationID

TLC Taxi Zone in which the taximeter was disengaged

In [33]:
# Checking the value counts
taxi_trips['DOLocationID'].value_counts()

74     996733
42     963534
41     858422
7      817099
129    751496
        ...  
2         165
99         45
199        13
105         8
110         1
Name: DOLocationID, Length: 263, dtype: int64

In [34]:
# How many null values are there in DOLocationID?
print("There are {} null values in DOLocationID".format(taxi_trips['DOLocationID'].isnull().sum()))

There are 0 null values in DOLocationID


In [35]:
# Checking the lowest and highest numbers in DOLocationID
print(taxi_trips['DOLocationID'].min())
print(taxi_trips['DOLocationID'].max())

1
265


In [36]:
# Merge DOLocationID with taxi_zones
taxi_trips = taxi_trips.merge(right=taxi_zones, how='left', left_on='DOLocationID', right_on='LocationID')

In [37]:
# Dropping LocationID from taxi_trips that was merged from taxi_zones
taxi_trips.drop(columns='LocationID', inplace=True)

In [38]:
# Renaming merged columns
taxi_trips.rename(columns={'Borough' : 'DO_Borough', 'Zone' : 'DO_Zone', 'service_zone' : 'DO_service_zone'}, inplace=True)

### passenger_count

The number of passengers in the vehicle (this is a driver entered value)

In [39]:
# Checking the value counts
taxi_trips['passenger_count'].value_counts()

1    23244224
2     2117741
0      971300
5      909801
6      468953
3      455954
4      156794
8         596
7         538
9         170
Name: passenger_count, dtype: int64

In [40]:
# Printing the number of missing passenger_count
print("There are {} records with missing passenger_count".format(taxi_trips.loc[taxi_trips['passenger_count']==0].shape[0]))

There are 971300 records with missing passenger_count


We will look at the possibility of populating the 0s in passenger_count in the next section.

### trip_distance

The elapsed trip distance in miles reported by the taximeter

In [41]:
# Checking the value counts
taxi_trips['trip_distance'].value_counts()

 0.00     438914
 0.90     295293
 1.00     292823
 0.80     289703
 1.10     276241
           ...  
-23.85         1
 52.57         1
 65.32         1
 74.03         1
 36.83         1
Name: trip_distance, Length: 8314, dtype: int64

In [42]:
# Checking the column description
taxi_trips['trip_distance'].describe()

count    2.832607e+07
mean     3.866140e+00
std      2.969125e+02
min     -2.032908e+04
25%      1.030000e+00
50%      1.860000e+00
75%      3.660000e+00
max      2.056541e+05
Name: trip_distance, dtype: float64

Minimum value of trip_distance is in the negative. Let's investigate the negatives

In [43]:
# How many rows are there where the trip_distance is in the negative?
print('There are {} records where the trip_distance is in the negative'.format(taxi_trips.loc[taxi_trips['trip_distance']<0].shape[0]))

There are 19487 records where the trip_distance is in the negative


In [44]:
# Checking the first 10 rows where the trip_distance is in the negative.
taxi_trips.loc[taxi_trips['trip_distance']<0].head(10)

,VendorID,lpep_pickup_datetime,lpep_dropoff_datetime,store_and_fwd_flag,RatecodeID,PULocationID,DOLocationID,passenger_count,trip_distance,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,payment_type,trip_type,congestion_surcharge,Vendor,Ratecode,PU_Borough,PU_Zone,PU_service_zone,DO_Borough,DO_Zone,DO_service_zone
21880092,2,2019-11-20 16:02:33,2019-11-20 16:23:41,N,5,197,124,1,-4.46,14.96,0.0,0.5,0.0,0.0,0.3,15.76,2,1,0.0,Verifone Inc.,Negotiated fare,Queens,Richmond Hill,Boro Zone,Queens,Howard Beach,Boro Zone
21880097,2,2019-11-21 17:39:56,2019-11-21 18:07:10,N,5,65,225,1,-3.87,23.51,0.0,0.5,0.0,0.0,0.3,24.31,2,1,0.0,Verifone Inc.,Negotiated fare,Brooklyn,Downtown Brooklyn/MetroTech,Boro Zone,Brooklyn,Stuyvesant Heights,Boro Zone
21880102,2,2019-11-23 08:25:54,2019-11-23 08:41:05,N,5,92,9,1,-5.28,18.95,0.0,0.5,0.0,0.0,0.3,19.75,2,1,0.0,Verifone Inc.,Negotiated fare,Queens,Flushing,Boro Zone,Queens,Auburndale,Boro Zone
21880105,2,2019-11-24 08:17:53,2019-11-24 08:41:19,N,5,188,225,1,-4.33,13.92,0.0,0.5,0.0,0.0,0.3,14.72,2,1,0.0,Verifone Inc.,Negotiated fare,Brooklyn,Prospect-Lefferts Gardens,Boro Zone,Brooklyn,Stuyvesant Heights,Boro Zone
21880109,2,2019-11-25 08:27:44,2019-11-25 08:43:54,N,5,177,35,1,-1.43,18.92,0.0,0.5,0.0,0.0,0.3,19.72,2,1,0.0,Verifone Inc.,Negotiated fare,Brooklyn,Ocean Hill,Boro Zone,Brooklyn,Brownsville,Boro Zone
21880110,2,2019-11-25 09:46:59,2019-11-25 10:55:47,N,5,35,48,1,-12.95,40.10,0.0,0.5,0.0,0.0,0.3,40.90,2,1,0.0,Verifone Inc.,Negotiated fare,Brooklyn,Brownsville,Boro Zone,Manhattan,Clinton East,Yellow Zone
21880113,2,2019-11-25 12:31:15,2019-11-25 14:13:59,N,5,217,48,1,-3.46,12.35,0.0,0.5,0.0,0.0,0.3,13.15,2,1,0.0,Verifone Inc.,Negotiated fare,Brooklyn,South Williamsburg,Boro Zone,Manhattan,Clinton East,Yellow Zone
21880114,2,2019-11-25 12:08:28,2019-11-25 14:51:40,N,5,91,108,1,-1.80,7.20,0.0,0.5,0.0,0.0,0.3,8.00,2,1,0.0,Verifone Inc.,Negotiated fare,Brooklyn,Flatlands,Boro Zone,Brooklyn,Gravesend,Boro Zone
21880115,2,2019-11-25 19:09:26,2019-11-25 19:57:56,N,5,33,76,1,-7.55,21.09,0.0,0.5,0.0,0.0,0.3,21.89,2,1,0.0,Verifone Inc.,Negotiated fare,Brooklyn,Brooklyn Heights,Boro Zone,Brooklyn,East New York,Boro Zone
21880175,2,2019-12-08 13:40:04,2019-12-08 14:01:43,N,5,124,76,1,-3.93,9.00,0.0,0.5,0.0,0.0,0.3,9.80,2,1,0.0,Verifone Inc.,Negotiated fare,Queens,Howard Beach,Boro Zone,Brooklyn,East New York,Boro Zone


These are valid records and looks like the minus is displayed in error

In [45]:
# Sum of trip_distance as it is
sum_trip_distance = taxi_trips['trip_distance'].sum()

# Sum of positive values in trip_distance
sum_trip_distance_positive = taxi_trips.loc[taxi_trips['trip_distance']>0,'trip_distance'].sum()

# Sum of negative values in trip_distance
sum_trip_distance_negative = taxi_trips.loc[taxi_trips['trip_distance']<0,'trip_distance'].sum()

# Expected sum of trip_distance when the negative values are converted to positive values
sum_trip_distance_exp_converted = taxi_trips.loc[taxi_trips['trip_distance']>0,'trip_distance'].sum() + taxi_trips.loc[taxi_trips['trip_distance']<0,'trip_distance'].abs().sum()

print('Sum of trip_distance: {:.2f}'.format(sum_trip_distance))
print('Sum of positive values in trip_distance: {:.2f}'.format(sum_trip_distance_positive))
print('Sum of negative values in trip_distance: {:.2f}'.format(sum_trip_distance_negative))
print('Expected sum after converting negative values to positive: {:.2f}'.format(sum_trip_distance_exp_converted))

Sum of trip_distance: 109512570.19
Sum of positive values in trip_distance: 109715069.74
Sum of negative values in trip_distance: -202499.55
Expected sum after converting negative values to positive: 109917569.29


In [46]:
# Reassigning the column with it's absolute value
taxi_trips['trip_distance'] = taxi_trips['trip_distance'].abs()

# Checking if the sum matches with the expected sum in the previous step
print('Actual sum of trip_distance after converting negative values to positive: {:.2f}'.format(taxi_trips['trip_distance'].sum()))

Actual sum of trip_distance after converting negative values to positive: 109917569.29


In [47]:
# Checking the column description
taxi_trips['trip_distance'].describe()

count    2.832607e+07
mean     3.880438e+00
std      2.969123e+02
min      0.000000e+00
25%      1.030000e+00
50%      1.870000e+00
75%      3.660000e+00
max      2.056541e+05
Name: trip_distance, dtype: float64

Minimum value of trip_distance is now zero. Why are there zeros?

In [48]:
# How many rows are there where the trip_distance is zero?
print('There are {} records where the trip_distance is zero'.format(taxi_trips.loc[taxi_trips['trip_distance']==0].shape[0]))

There are 438914 records where the trip_distance is zero


Possible explanation of zero trip_distance:
 - Ride was less than 0.1 miles. This can be identified if the minimum fare is provided
 - Driver forgot to engage the meter.
 
We will look at the possibility of populating the 0s in trip_distance in the next section.

### fare_amount

The time-and-distance fare calculated by the meter

In [49]:
# Checking the column description
taxi_trips['fare_amount'].describe()

count    2.832607e+07
mean     1.318245e+01
std      1.187243e+01
min     -8.900000e+02
25%      6.500000e+00
50%      9.500000e+00
75%      1.600000e+01
max      1.044584e+04
Name: fare_amount, dtype: float64

In [50]:
# Reassigning the column with it's absolute value
taxi_trips['fare_amount'] = taxi_trips['fare_amount'].abs()

In [51]:
# Checking the column description
taxi_trips['fare_amount'].describe()

count    2.832607e+07
mean     1.321725e+01
std      1.183367e+01
min      0.000000e+00
25%      6.500000e+00
50%      9.500000e+00
75%      1.600000e+01
max      1.044584e+04
Name: fare_amount, dtype: float64

In [52]:
# How many rows are there where the fare_amount is zero?
fare_amount_zero = taxi_trips.loc[taxi_trips['fare_amount']==0].shape[0]

print('There are {} records where the fare_amount is zero'.format(fare_amount_zero))

There are 59225 records where the fare_amount is zero


We will look at the possibility of populating the 0s in fare_amount in the next section. From running describe(), we can see that the maximum fare_amount is 10445.84. This seems highly unlikely and we will investigate values like this in the next section.

### extra

Miscellaneous extras and surcharges (this only includes the 0.50 and 1.00 rush hour and overnight charges)

In [53]:
# Checking the column description
taxi_trips['extra'].describe()

count    2.832607e+07
mean     3.957281e-01
std      5.724350e-01
min     -2.542000e+01
25%      0.000000e+00
50%      0.000000e+00
75%      5.000000e-01
max      3.000000e+01
Name: extra, dtype: float64

In [54]:
# Reassigning the column with it's absolute value
taxi_trips['extra'] = taxi_trips['extra'].abs()

In [55]:
# Checking the value counts
taxi_trips['extra'].value_counts()

0.00    14389099
0.50     8158890
1.00     5066194
2.75      605657
3.25       35476
          ...   
1.46           1
2.20           1
3.66           1
1.71           1
0.83           1
Name: extra, Length: 105, dtype: int64

surcharges only includes the charges of 0.50 and 1.00 but it's possible to have other values as the Miscellaneous extras could be anything other than negatives. 0s are fine for this column.

### mta_tax

$0.50 MTA tax that is automatically triggered based on the metered rate in use

In [56]:
taxi_trips['mta_tax'].describe()

count    2.832607e+07
mean     4.785649e-01
std      1.085238e-01
min     -5.000000e-01
25%      5.000000e-01
50%      5.000000e-01
75%      5.000000e-01
max      8.291000e+01
Name: mta_tax, dtype: float64

In [57]:
# Reassigning the column with it's absolute value
taxi_trips['mta_tax'] = taxi_trips['mta_tax'].abs()

In [58]:
taxi_trips['mta_tax'].value_counts()

0.50     27250099
0.00      1075875
3.55           42
0.08            7
0.60            7
0.11            6
0.55            5
0.07            5
0.25            5
0.09            2
0.12            1
17.33           1
0.13            1
0.36            1
1.37            1
0.10            1
0.65            1
1.00            1
82.91           1
4.41            1
0.81            1
2.75            1
26.22           1
0.32            1
0.18            1
3.80            1
1.50            1
0.83            1
Name: mta_tax, dtype: int64

Only 0.00 and 0.50 are acceptable for this column. We will investigate other values in the next section.

### tip_amount

Tip amount (automatically populated for credit card tips - cash tips are not included)

In [59]:
taxi_trips['tip_amount'].describe()

count    2.832607e+07
mean     1.080043e+00
std      2.094613e+00
min     -1.010000e+02
25%      0.000000e+00
50%      0.000000e+00
75%      1.860000e+00
max      6.412000e+02
Name: tip_amount, dtype: float64

In [60]:
# Reassigning the column with it's absolute value
taxi_trips['tip_amount'] = taxi_trips['tip_amount'].abs()

In [61]:
taxi_trips['tip_amount'].value_counts()

0.00      17264828
1.00        812133
2.00        595508
2.75        289056
3.00        228955
            ...   
94.29            1
46.92            1
30.83            1
32.17            1
224.57           1
Name: tip_amount, Length: 4058, dtype: int64

tip_amount could be any number. However, from the above value counts, we can see that there's one tip amount of 224.57. This seems highly unlikely and we will investigate values like this in the next section.

### tolls_amount

Total amount of all tolls paid in trip

In [62]:
taxi_trips['tolls_amount'].describe()

count    2.832607e+07
mean     1.764550e-01
std      1.988191e+00
min     -8.000000e+01
25%      0.000000e+00
50%      0.000000e+00
75%      0.000000e+00
max      7.999920e+03
Name: tolls_amount, dtype: float64

In [63]:
# Reassigning the column with it's absolute value
taxi_trips['tolls_amount'] = taxi_trips['tolls_amount'].abs()

In [64]:
taxi_trips['tolls_amount'].value_counts()

0.00     27499507
5.76       429773
6.12       238267
5.54        39120
2.64        27228
           ...   
6.69            1
56.20           1
11.42           1
4.03            1
3.20            1
Name: tolls_amount, Length: 1329, dtype: int64

tolls_amount could be any number. However, from running describe(), we can see that the maximum tolls_amount is 7999.92. This seems highly unlikely and we will investigate values like this in the next section.

### improvement_surcharge

$0.30 improvement surcharge assessed on hailed trips at the flag drop

In [65]:
taxi_trips['improvement_surcharge'].describe()

count    2.832607e+07
mean     2.869971e-01
std      6.460169e-02
min     -3.000000e-01
25%      3.000000e-01
50%      3.000000e-01
75%      3.000000e-01
max      4.700000e-01
Name: improvement_surcharge, dtype: float64

In [66]:
# Reassigning the column with it's absolute value
taxi_trips['improvement_surcharge'] = taxi_trips['improvement_surcharge'].abs()

In [67]:
taxi_trips['improvement_surcharge'].value_counts()

0.30    27237306
0.00     1088751
0.43           3
0.11           2
0.12           1
0.46           1
0.32           1
0.44           1
0.47           1
0.17           1
0.13           1
Name: improvement_surcharge, dtype: int64

In [68]:
# Checking the number of null values
taxi_trips['improvement_surcharge'].isnull().sum()

2

In [69]:
# Filling the null values with 0.0
taxi_trips['improvement_surcharge'].fillna(value=0.0, inplace=True)

Only 0.0 and 0.3 are acceptable for improvement_surcharge. We will investigate other values in the next section.

### total_amount

The total amount charged to passengers (does not include cash tips)

In [70]:
taxi_trips['total_amount'].describe()

count    2.832607e+07
mean     1.571784e+01
std      1.321935e+01
min     -8.903000e+02
25%      8.300000e+00
50%      1.180000e+01
75%      1.880000e+01
max      1.052875e+04
Name: total_amount, dtype: float64

In [71]:
# Reassigning the column with it's absolute value
taxi_trips['total_amount'] = taxi_trips['total_amount'].abs()

In [72]:
# How many rows are there where the total_amount is zero?
total_amount_zero = taxi_trips.loc[taxi_trips['total_amount']==0].shape[0]

print('There are {} records where the total_amount is zero'.format(total_amount_zero))

There are 56972 records where the total_amount is zero


We will look at the possibility of populating the 0s in total_amount in the next section.

### payment_type

A numeric code signifying how the passenger paid for the trip (1= Credit card; 2= Cash; 3= No charge; 4= Dispute; 5= Unknown; 6= Voided trip)

In [73]:
# Checking values in payment_type
taxi_trips['payment_type'].value_counts()

1    14793538
2    12392982
0      942199
3      132220
4       63966
5        1166
Name: payment_type, dtype: int64

In [74]:
# Dictionary of code and name
payment_types = {1 : 'Credit card', 2 : 'Cash', 3 : 'No charge', 4 : 'Dispute', 5 : 'Unknown', 6 : 'Voided trip', 0 : 'Unknown'}

# Mapping the payment type
taxi_trips['payment_type'] = taxi_trips['payment_type'].map(payment_types)

taxi_trips['payment_type'].value_counts()

Credit card    14793538
Cash           12392982
Unknown          943365
No charge        132220
Dispute           63966
Name: payment_type, dtype: int64

There are 943365 unknown payment_type. We will look at the possibility of populating them in the next section.

### trip_type

A code indicating whether the trip was a street-hail or a dispatch that is automatically assigned based on the metered rate in use but can be altered by the driver (1= Street-hail; 2= Dispatch)

In [75]:
# Checking values in trip_type
taxi_trips['trip_type'].value_counts()

1    26640078
0      942653
2      743340
Name: trip_type, dtype: int64

In [76]:
trip_types = {1 : 'Street-hail', 2 : 'Dispatch', 0 : 'Unknown'}

taxi_trips['trip_type'] = taxi_trips['trip_type'].map(trip_types)

taxi_trips['trip_type'].value_counts()

Street-hail    26640078
Unknown          942653
Dispatch         743340
Name: trip_type, dtype: int64

There are 942653 unknown trip_type. We will look at the possibility of populating them in the next section.

### congestion_surcharge

Congestion surcharge for trips that start, end or pass through the congestion zone in Manhattan, south of 96th street (2.50 for non-shared trips in Yellow Taxis; 2.75 for non-shared trips in Green Taxis)

In [77]:
taxi_trips['congestion_surcharge'].describe()

count    2.683749e+07
mean     9.726175e-02
std      5.078660e-01
min     -2.750000e+00
25%      0.000000e+00
50%      0.000000e+00
75%      0.000000e+00
max      2.750000e+00
Name: congestion_surcharge, dtype: float64

In [78]:
# Reassigning the column with it's absolute value
taxi_trips['congestion_surcharge'] = taxi_trips['congestion_surcharge'].abs()

In [79]:
taxi_trips['congestion_surcharge'].value_counts()

0.00    25887588
2.75      944457
2.50        5361
0.75          81
2.00           2
0.30           1
Name: congestion_surcharge, dtype: int64

In [80]:
# Checking the number of null values
taxi_trips['congestion_surcharge'].isnull().sum()

1488581

In [81]:
# Filling the null values with 0.0
taxi_trips['congestion_surcharge'].fillna(value=0.0, inplace=True)

Only 0.00, 2.50, and 2.75 are valid for congestion_surcharge. We will investigate other values in the next section.

# Cleaning Unrecognised and Unknown Values

In this section, we will try to update unknown values and clean unrecognised values we came across in the previous section.

### New column: total_amount_check

For checking purposes, we are going to create a new column named total_amount_check with the below formula:
- total_amount = fare_amount + improvement_surcharge + tolls_amount + tip_amount + mta_tax + extra + congestion_surcharge

In [82]:
# Creating new column total_amount_check with the above formula
taxi_trips['total_amount_check'] = (taxi_trips['fare_amount'] + taxi_trips['improvement_surcharge'] + taxi_trips['tolls_amount'] + taxi_trips['tip_amount'] + taxi_trips['mta_tax'] + taxi_trips['extra'] + taxi_trips['congestion_surcharge']).round(2)

To check the validity of our formula, we need to see if the total_amount_check actually matches with total_amount.

In [83]:
# Creating a conditional column to see if the totals match
taxi_trips['total_amount_match?'] = np.where(taxi_trips['total_amount'] == taxi_trips['total_amount_check'], 'Match', 'No Match')

In [84]:
taxi_trips['total_amount_match?'].value_counts()

Match       27735546
No Match      590525
Name: total_amount_match?, dtype: int64

97.9% of values in total_amount_check matches with total_amount. This shows the formula is correct and for the values that don't match, we will need to carry out further invetsigation.

In [85]:
# Creating a new column to hold the difference between total_amount and total_amount_check
taxi_trips['total_amount_difference'] = taxi_trips['total_amount'] - taxi_trips['total_amount_check']

### New column: trip_duration_mins

Creating a new column to hold trip duration in minutes. This could especially be useful in updating the 0s in fare_amount whilst determining the validity of high values in fare_amount

In [86]:
taxi_trips['trip_duration_mins'] = (taxi_trips['lpep_dropoff_datetime'] - taxi_trips['lpep_pickup_datetime']) / np.timedelta64(1, 'm')

### congestion_surcharge

Only 0.00, 2.50, and 2.75 are valid for congestion_surcharge. Others needs to be modified.

In [87]:
taxi_trips['congestion_surcharge'].value_counts()

0.00    27376169
2.75      944457
2.50        5361
0.75          81
2.00           2
0.30           1
Name: congestion_surcharge, dtype: int64

Let's check 0.3 first as it is only one record. It should be either 0.00, 2.50 or 2.75

In [88]:
# Rows where congestion_surcharge is 0.3
taxi_trips.loc[taxi_trips['congestion_surcharge']==0.3, ['fare_amount','extra','mta_tax','tip_amount','tolls_amount','improvement_surcharge','congestion_surcharge','total_amount','total_amount_check','total_amount_difference']]

,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,congestion_surcharge,total_amount,total_amount_check,total_amount_difference
21224506,161.0,0.0,0.5,32.35,0.0,0.3,0.3,194.15,194.45,-0.3


From the above, we can conclude that the 0.3 congestion_surcharge is an error and should be 0.00 as the total_amount is 0.30 less than the total_amount_check

In [89]:
# Replacing 0.3 in congestion_surcharge with 0.00
taxi_trips.loc[taxi_trips['congestion_surcharge']==0.3, 'congestion_surcharge'] = 0.00

Now, let's check 2.0. It should be either 0.00, 2.50 or 2.75

In [90]:
# Rows where congestion_surcharge is 2.0 and total_amount_difference is 0.75
taxi_trips.loc[(taxi_trips['congestion_surcharge']==2.0) & (taxi_trips['total_amount_difference']==0.75), ['fare_amount','extra','mta_tax','tip_amount','tolls_amount','improvement_surcharge','congestion_surcharge','total_amount','total_amount_check','total_amount_difference']]

,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,congestion_surcharge,total_amount,total_amount_check,total_amount_difference
25940904,7.0,0.5,0.5,0.0,0.0,0.3,2.0,11.05,10.3,0.75
25940906,6.5,0.5,0.5,0.0,0.0,0.3,2.0,10.55,9.8,0.75


From the above, we can conclude that the 2.0 congestion_surcharge is an error and should be 2.75 as the total_amount is 0.75 more than the total_amount_check

In [91]:
# Replacing 2.0 in congestion_surcharge with 2.75 where congestion_surcharge is 2.0 and total_amount_difference is 0.75
taxi_trips.loc[(taxi_trips['congestion_surcharge']==2.0) & (taxi_trips['total_amount_difference']==0.75), 'congestion_surcharge'] = 2.75

Now, let's check 0.75. It should be either 0.00, 2.50 or 2.75

In [92]:
# Rows where congestion_surcharge is 0.75 and total_amount_difference is 2.00
taxi_trips.loc[(taxi_trips['congestion_surcharge']==0.75) & ((taxi_trips['total_amount_difference']==2.00) | (taxi_trips['total_amount_difference']==1.75)), ['fare_amount','extra','mta_tax','tip_amount','tolls_amount','improvement_surcharge','congestion_surcharge','total_amount','total_amount_check','total_amount_difference']]

,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,congestion_surcharge,total_amount,total_amount_check,total_amount_difference


There are no records where the total_amount_difference is 2.00 or 1.75 when the congestion_surcharge is 0.75. This might be due to rounding. Let's check what values are in total_amount_difference when the congestion_surcharge is 0.75

In [93]:
# Checking the value counts of total_amount_difference when the congestion_surcharge is 0.75
taxi_trips.loc[taxi_trips['congestion_surcharge']==0.75, 'total_amount_difference'].value_counts()

0.00    72
1.95     7
1.95     1
9.90     1
Name: total_amount_difference, dtype: int64

There are 8 records where the total_amount_difference becomes 2 when rounded to 0 decimal place. We can update them as 2.75

In [95]:
# Replacing 0.75 in congestion_surcharge with 2.75 where total_amount_difference is 2.0
taxi_trips.loc[(taxi_trips['congestion_surcharge']==0.75) & (taxi_trips['total_amount_difference'].round(0)==2.00), 'congestion_surcharge'] = 2.75

We are low left with 73 records where congestion_surcharge is 0.75:
 - total_amount_difference is 0 for 72 of those records. As this congestion_surcharge of 0.75 is included in the total_amount, we should keep this somewhere but not in congestion_surcharge as only 0.00, 2.50 and 2.75 are valid, and setting the congestion_surcharge to either 0, 2.50 or 2.75 would cause issues for the total_amount.
 - total_amount_difference is 9.90 for 1 record. total_amount is off by 9.90 and updating congestion_surcharge as 2.75 will bring the difference down to 7.90
 
We can move these amounts to another column that's more suitable.

In [96]:
taxi_trips['congestion_surcharge'].value_counts()

0.00    27376170
2.75      944467
2.50        5361
0.75          73
Name: congestion_surcharge, dtype: int64

In [97]:
# Checking the sums before updating the values

sum_congestion_surcharge = taxi_trips['congestion_surcharge'].sum()
sum_extra = taxi_trips['extra'].sum()
sum_values_to_move = taxi_trips.loc[taxi_trips['congestion_surcharge']==0.75, 'congestion_surcharge'].sum()
exp_sum_congestion_surcharge = taxi_trips['congestion_surcharge'].sum() - sum_values_to_move
exp_sum_extra = sum_extra + sum_values_to_move

print('Sum of congestion_surcharge: {:.2f}'.format(sum_congestion_surcharge))
print('Sum of extra: {:.2f}'.format(sum_extra))
print('Sum of values to be moved: {:.2f}'.format(sum_values_to_move))
print('Expected sum of congestion_surcharge after update: {:.2f}'.format(exp_sum_congestion_surcharge))
print('Expected sum of extra after update: {:.2f}'.format(exp_sum_extra))

Sum of congestion_surcharge: 2610741.50
Sum of extra: 11259442.63
Sum of values to be moved: 54.75
Expected sum of congestion_surcharge after update: 2610686.75
Expected sum of extra after update: 11259497.38


In [99]:
# Adding 0.75 values in congestion_surcharge to extra
taxi_trips.loc[taxi_trips['congestion_surcharge']==0.75, 'extra'] = taxi_trips['extra'] + taxi_trips['congestion_surcharge']

# Setting 0.75 values in congestion_surcharge as 0.00
taxi_trips.loc[taxi_trips['congestion_surcharge']==0.75, 'congestion_surcharge'] = 0.00

In [100]:
# Checking the sums after updating the values to see if they are as expected

sum_congestion_surcharge = taxi_trips['congestion_surcharge'].sum()
sum_extra = taxi_trips['extra'].sum()

print('Sum of congestion_surcharge: {:.2f}'.format(sum_congestion_surcharge))
print('Sum of extra: {:.2f}'.format(sum_extra))

Sum of congestion_surcharge: 2610686.75
Sum of extra: 11259497.38


The sums matches with the expected sums.

In [101]:
# Checking the values in congestion_surcharge for confirmation
taxi_trips['congestion_surcharge'].value_counts()

0.00    27376243
2.75      944467
2.50        5361
Name: congestion_surcharge, dtype: int64

The congestion_surcharge column is now clean with values of 0.00, 2.50 and 2.75 only

### improvement_surcharge

$0.30 improvement surcharge assessed on hailed trips at the flag drop

In [102]:
taxi_trips['improvement_surcharge'].value_counts()

0.30    27237306
0.00     1088753
0.43           3
0.11           2
0.12           1
0.46           1
0.32           1
0.44           1
0.47           1
0.17           1
0.13           1
Name: improvement_surcharge, dtype: int64

improvement_surcharge should either be 0.00 or 0.30. Let's investigate values that are not 0.00 or 0.30

In [103]:
# Checking the rows where the improvement_surcharge is not 0.30 or 0.00
taxi_trips.loc[~(taxi_trips['improvement_surcharge']==0.30) & ~(taxi_trips['improvement_surcharge']==0.00)]

,VendorID,lpep_pickup_datetime,lpep_dropoff_datetime,store_and_fwd_flag,RatecodeID,PULocationID,DOLocationID,passenger_count,trip_distance,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,payment_type,trip_type,congestion_surcharge,Vendor,Ratecode,PU_Borough,PU_Zone,PU_service_zone,DO_Borough,DO_Zone,DO_service_zone,total_amount_check,total_amount_match?,total_amount_difference,trip_duration_mins
18577533,2,2018-10-30 07:39:44,2018-10-30 07:39:46,N,1,265,265,1,0.0,3.50,0.0,0.0,0.0,0.00,0.11,3.61,Cash,Street-hail,0.0,Verifone Inc.,Standard rate,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,3.61,Match,0.0,0.033333
18577534,2,2018-10-30 07:40:11,2018-10-30 07:40:23,N,1,265,265,1,0.0,3.50,0.0,0.0,0.0,0.00,0.11,3.61,Cash,Street-hail,0.0,Verifone Inc.,Standard rate,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,3.61,Match,0.0,0.200000
22075459,2,2019-11-12 15:32:34,2019-11-12 15:33:24,N,1,265,265,2,0.0,3.73,0.0,0.0,0.0,0.00,0.44,4.17,Cash,Street-hail,0.0,Verifone Inc.,Standard rate,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,4.17,Match,0.0,0.833333
22086638,2,2019-11-12 13:54:01,2019-11-12 13:54:51,N,1,265,265,1,0.0,3.50,0.0,0.0,0.0,0.00,0.43,3.93,Cash,Street-hail,0.0,Verifone Inc.,Standard rate,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,3.93,Match,0.0,0.833333
22086640,2,2019-11-12 16:00:12,2019-11-12 16:01:52,N,1,265,265,1,0.0,3.96,0.0,0.0,0.0,0.00,0.12,4.08,Cash,Street-hail,0.0,Verifone Inc.,Standard rate,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,4.08,Match,0.0,1.666667
22090530,2,2019-12-03 17:40:00,2019-12-03 17:40:28,N,1,193,193,1,0.0,3.50,2.0,0.0,0.0,0.00,0.17,5.67,Cash,Street-hail,0.0,Verifone Inc.,Standard rate,Queens,Queensbridge/Ravenswood,Boro Zone,Queens,Queensbridge/Ravenswood,Boro Zone,5.67,Match,0.0,0.466667
22362775,2,2019-11-12 15:29:51,2019-11-12 15:31:42,N,1,265,265,1,0.0,4.42,0.0,0.0,0.0,0.28,0.46,5.16,Cash,Street-hail,0.0,Verifone Inc.,Standard rate,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,5.16,Match,0.0,1.850000
22362776,2,2019-11-12 15:57:44,2019-11-12 15:59:44,N,1,265,265,1,0.0,4.19,0.0,0.0,0.0,4.50,0.13,8.82,Cash,Street-hail,0.0,Verifone Inc.,Standard rate,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,8.82,Match,0.0,2.000000
23029537,2,2019-11-08 10:42:12,2019-11-08 14:30:21,N,1,265,264,1,0.0,84.90,0.0,0.0,0.0,0.00,0.32,85.22,Cash,Street-hail,0.0,Verifone Inc.,Standard rate,Unknown,Unknown,Unknown,Unknown,NV,Unknown,85.22,Match,0.0,228.150000
23031493,2,2019-11-08 10:42:12,2019-11-08 14:34:55,N,1,265,264,4,0.0,3.50,0.0,0.0,0.0,0.00,0.43,3.93,Cash,Street-hail,0.0,Verifone Inc.,Standard rate,Unknown,Unknown,Unknown,Unknown,NV,Unknown,3.93,Match,0.0,232.716667


In [104]:
# Checking if there's any link between improvement_surcharge and trip_type using pivot table
pd.pivot_table(taxi_trips, index='trip_type', columns='improvement_surcharge', values='lpep_pickup_datetime', aggfunc='count', fill_value=0, margins=True, margins_name='Total')

improvement_surcharge,0.0,0.11,0.12,0.13,0.17,0.3,0.32,0.43,0.44,0.46,0.47,Total
trip_type,,,,,,,,,,,,
Dispatch,709701,0,0,0,0,33639,0,0,0,0,0,743340
Street-hail,30255,2,1,1,1,26609811,1,3,1,1,1,26640078
Unknown,348797,0,0,0,0,593856,0,0,0,0,0,942653
Total,1088753,2,1,1,1,27237306,1,3,1,1,1,28326071


In [105]:
# Checking if there's any link between improvement_surcharge and Ratecode using pivot table
pd.pivot_table(taxi_trips, index='Ratecode', columns='improvement_surcharge', values='lpep_pickup_datetime', aggfunc='count', fill_value=0, margins=True, margins_name='Total')

improvement_surcharge,0.0,0.11,0.12,0.13,0.17,0.3,0.32,0.43,0.44,0.46,0.47,Total
Ratecode,,,,,,,,,,,,
Group ride,4,0,0,0,0,345,0,0,0,0,0,349
JFK,421,0,0,0,0,55113,0,0,0,0,0,55534
Nassau or Westchester,274,0,0,0,0,18224,0,0,0,0,0,18498
Negotiated fare,687459,0,0,0,0,52763,0,0,0,0,0,740222
Newark,97,0,0,0,0,13672,0,0,0,0,0,13769
Standard rate,51654,2,1,1,1,26503680,1,3,1,1,1,26555346
Unknown,348844,0,0,0,0,593509,0,0,0,0,0,942353
Total,1088753,2,1,1,1,27237306,1,3,1,1,1,28326071


In [106]:
# Checking if there's any link between improvement_surcharge and payment_type using pivot table
pd.pivot_table(taxi_trips, index='payment_type', columns='improvement_surcharge', values='lpep_pickup_datetime', aggfunc='count', fill_value=0, margins=True, margins_name='Total')

improvement_surcharge,0.0,0.11,0.12,0.13,0.17,0.3,0.32,0.43,0.44,0.46,0.47,Total
payment_type,,,,,,,,,,,,
Cash,288939,2,1,1,1,12104031,1,3,1,1,1,12392982
Credit card,439141,0,0,0,0,14354397,0,0,0,0,0,14793538
Dispute,4232,0,0,0,0,59734,0,0,0,0,0,63966
No charge,7509,0,0,0,0,124711,0,0,0,0,0,132220
Unknown,348932,0,0,0,0,594433,0,0,0,0,0,943365
Total,1088753,2,1,1,1,27237306,1,3,1,1,1,28326071


From above tables, the reason for having values other than 0.00 and 0.30 are not obvious. But all 12 occured with street-hail with standard rate where the payment_type is cash. From above pivot tables, the following are clear:
 - 99.9% of street-hail	trips pays 0.30 improvement_surcharge
 - 99.8% of standard rate trips pays 0.30 improvement_surcharge

These charges are included in the total amount passengers are charged, which is confirmed by the total_amount_difference. Therefore, setting the improvement_surcharge to either 0 or 0.30 would cause issues for the total_amount.

One option would be to move these amounts to another column that's more suitable. Two reasons for this are:
 - Amount isn't 0.30. Therefore, this might not even be improvement_surcharge.
 - It is included in the total_amount. Therefore, we need to keep it. Otherwise, the total_amount the passenger was charged wouldn't add up.

Let's move the values other than 0.3 to extra as it seems the most suitable category.

In [107]:
# Checking the sums before updating the values

sum_improvement_surcharge = taxi_trips['improvement_surcharge'].sum()
sum_extra = taxi_trips['extra'].sum()
sum_values_to_move = taxi_trips.loc[~(taxi_trips['improvement_surcharge']==0.30) & ~(taxi_trips['improvement_surcharge']==0.00), 'improvement_surcharge'].sum()
exp_sum_improvement_surcharge = taxi_trips['improvement_surcharge'].sum() - sum_values_to_move
exp_sum_extra = sum_extra + sum_values_to_move

print('Sum of improvement_surcharge: {:.2f}'.format(sum_improvement_surcharge))
print('Sum of extra: {:.2f}'.format(sum_extra))
print('Sum of values to be moved: {:.2f}'.format(sum_values_to_move))
print('Expected sum of improvement_surcharge after update: {:.2f}'.format(exp_sum_improvement_surcharge))
print('Expected sum of extra after update: {:.2f}'.format(exp_sum_extra))

Sum of improvement_surcharge: 8171195.42
Sum of extra: 11259497.38
Sum of values to be moved: 3.62
Expected sum of improvement_surcharge after update: 8171191.80
Expected sum of extra after update: 11259501.00


In [108]:
# Adding values other than 0.3 to extra
taxi_trips.loc[~(taxi_trips['improvement_surcharge']==0.30) & ~(taxi_trips['improvement_surcharge']==0.00), 'extra'] = taxi_trips['extra'] + taxi_trips['improvement_surcharge']

# Setting values other than 0.3 as 0.00 in improvement_surcharge
taxi_trips.loc[~(taxi_trips['improvement_surcharge']==0.30) & ~(taxi_trips['improvement_surcharge']==0.00), 'improvement_surcharge'] = 0.00

In [109]:
# Checking the sums after updating the values to see if they are as expected

sum_improvement_surcharge = taxi_trips['improvement_surcharge'].sum()
sum_extra = taxi_trips['extra'].sum()

print('Sum of improvement_surcharge: {:.2f}'.format(sum_improvement_surcharge))
print('Sum of extra: {:.2f}'.format(sum_extra))

Sum of improvement_surcharge: 8171191.80
Sum of extra: 11259501.00


The sums matches with the expected sums.

In [110]:
# Checking the values in improvement_surcharge for confirmation
taxi_trips['improvement_surcharge'].value_counts()

0.3    27237306
0.0     1088765
Name: improvement_surcharge, dtype: int64

The improvement_surcharge column is now clean with values of 0.0 and 0.3 only

### mta_tax

$0.50 MTA tax that is automatically triggered based on the metered rate in use

In [111]:
taxi_trips['mta_tax'].value_counts()

0.50     27250099
0.00      1075875
3.55           42
0.08            7
0.60            7
0.11            6
0.55            5
0.07            5
0.25            5
0.09            2
0.12            1
17.33           1
0.13            1
0.36            1
1.37            1
0.10            1
0.65            1
1.00            1
82.91           1
4.41            1
0.81            1
2.75            1
26.22           1
0.32            1
0.18            1
3.80            1
1.50            1
0.83            1
Name: mta_tax, dtype: int64

Values in mta_tax should either be 0.00 or 0.50. Let's investigate values that are not 0.00 or 0.50

In [112]:
# Checking the rows where the mta_tax is not 0.50 or 0.00
taxi_trips.loc[~(taxi_trips['mta_tax']==0.50) & ~(taxi_trips['mta_tax']==0.00)]

,VendorID,lpep_pickup_datetime,lpep_dropoff_datetime,store_and_fwd_flag,RatecodeID,PULocationID,DOLocationID,passenger_count,trip_distance,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,payment_type,trip_type,congestion_surcharge,Vendor,Ratecode,PU_Borough,PU_Zone,PU_service_zone,DO_Borough,DO_Zone,DO_service_zone,total_amount_check,total_amount_match?,total_amount_difference,trip_duration_mins
492111,1,2017-01-12 15:29:58,2017-01-12 15:31:02,N,3,265,265,1,0.0,20.50,0.50,0.60,0.0,0.0,0.0,21.60,Cash,Dispatch,0.00,"Creative Mobile Technologies, LLC",Newark,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,21.60,Match,0.00,1.066667
492112,1,2017-01-12 15:29:12,2017-01-12 15:29:30,N,3,265,265,1,0.0,20.00,0.50,0.60,0.0,0.0,0.0,21.10,Cash,Street-hail,0.00,"Creative Mobile Technologies, LLC",Newark,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,21.10,Match,0.00,0.300000
492113,1,2017-01-12 15:27:28,2017-01-12 15:28:44,N,3,265,265,1,0.0,20.50,0.50,0.60,0.0,0.0,0.0,21.60,Cash,Street-hail,0.00,"Creative Mobile Technologies, LLC",Newark,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,21.60,Match,0.00,1.266667
492485,1,2017-01-12 14:49:33,2017-01-12 14:52:09,N,3,145,145,1,0.0,20.50,0.00,0.60,0.0,0.0,0.0,21.10,Cash,Street-hail,0.00,"Creative Mobile Technologies, LLC",Newark,Queens,Long Island City/Hunters Point,Boro Zone,Queens,Long Island City/Hunters Point,Boro Zone,21.10,Match,0.00,2.600000
493314,1,2017-01-12 14:52:46,2017-01-12 14:55:01,N,5,145,145,1,0.0,1.23,0.00,0.60,0.0,0.0,0.0,1.83,Cash,Street-hail,0.00,"Creative Mobile Technologies, LLC",Negotiated fare,Queens,Long Island City/Hunters Point,Boro Zone,Queens,Long Island City/Hunters Point,Boro Zone,1.83,Match,0.00,2.250000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27476246,1,2020-01-28 19:16:01,2020-01-28 19:16:07,N,1,232,232,1,2.3,13.00,3.75,3.55,0.0,0.0,0.0,20.30,Dispute,Street-hail,2.75,"Creative Mobile Technologies, LLC",Standard rate,Manhattan,Two Bridges/Seward Park,Yellow Zone,Manhattan,Two Bridges/Seward Park,Yellow Zone,23.05,No Match,-2.75,0.100000
27476247,1,2020-06-13 10:59:11,2020-06-13 11:04:10,N,1,236,75,1,0.6,5.00,3.75,3.55,0.0,0.0,0.0,12.30,Cash,Street-hail,2.75,"Creative Mobile Technologies, LLC",Standard rate,Manhattan,Upper East Side North,Yellow Zone,Manhattan,East Harlem South,Boro Zone,15.05,No Match,-2.75,4.983333
27480831,1,2020-01-01 10:58:27,2020-01-01 10:58:27,N,1,107,107,2,4.5,15.50,2.75,3.55,2.5,0.0,0.0,24.30,Credit card,Street-hail,2.75,"Creative Mobile Technologies, LLC",Standard rate,Manhattan,Gramercy,Yellow Zone,Manhattan,Gramercy,Yellow Zone,27.05,No Match,-2.75,0.000000
27522650,1,2020-01-18 14:13:47,2020-01-18 14:13:49,N,1,263,263,1,2.6,11.00,2.75,3.55,2.0,0.0,0.0,19.30,Credit card,Street-hail,2.75,"Creative Mobile Technologies, LLC",Standard rate,Manhattan,Yorkville West,Yellow Zone,Manhattan,Yorkville West,Yellow Zone,22.05,No Match,-2.75,0.033333


In [113]:
# Checking if there's any link between mta_tax and Ratecode using pivot table
pd.pivot_table(taxi_trips, index='Ratecode', columns='mta_tax', values='lpep_pickup_datetime', aggfunc='count', fill_value=0)

mta_tax,0.00,0.07,0.08,0.09,0.10,0.11,0.12,0.13,0.18,0.25,0.32,0.36,0.50,0.55,0.60,0.65,0.81,0.83,1.00,1.37,1.50,2.75,3.55,3.80,4.41,17.33,26.22,82.91
Ratecode,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Group ride,6,0,0,0,0,0,0,0,0,0,0,0,343,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
JFK,395,0,0,0,0,0,0,0,0,0,0,0,55139,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
Nassau or Westchester,270,0,0,0,0,0,0,0,0,0,0,0,18228,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
Negotiated fare,481912,0,0,0,0,0,0,0,0,0,0,0,258307,0,2,0,0,0,0,0,1,0,0,0,0,0,0,0
Newark,13764,0,0,0,0,0,0,0,0,0,0,0,0,0,5,0,0,0,0,0,0,0,0,0,0,0,0,0
Standard rate,49870,5,6,2,1,6,1,1,1,5,1,1,26505388,5,0,1,1,1,1,1,0,1,42,1,1,1,1,1
Unknown,529658,0,1,0,0,0,0,0,0,0,0,0,412694,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [114]:
# Checking if there's any link between mta_tax and Ratecode using pivot table
pd.pivot_table(taxi_trips, index='trip_type', columns='mta_tax', values='lpep_pickup_datetime', aggfunc='count', fill_value=0)

mta_tax,0.00,0.07,0.08,0.09,0.10,0.11,0.12,0.13,0.18,0.25,0.32,0.36,0.50,0.55,0.60,0.65,0.81,0.83,1.00,1.37,1.50,2.75,3.55,3.80,4.41,17.33,26.22,82.91
trip_type,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Dispatch,495813,0,0,0,0,0,0,0,0,0,0,0,247526,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
Street-hail,50431,5,7,2,1,6,1,1,1,5,1,1,26589551,5,6,1,1,1,1,1,1,1,42,1,1,1,1,1
Unknown,529631,0,0,0,0,0,0,0,0,0,0,0,413022,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [115]:
# Checking if there's any link between mta_tax and Ratecode using pivot table
pd.pivot_table(taxi_trips, index='payment_type', columns='mta_tax', values='lpep_pickup_datetime', aggfunc='count', fill_value=0)

mta_tax,0.00,0.07,0.08,0.09,0.10,0.11,0.12,0.13,0.18,0.25,0.32,0.36,0.50,0.55,0.60,0.65,0.81,0.83,1.00,1.37,1.50,2.75,3.55,3.80,4.41,17.33,26.22,82.91
payment_type,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Cash,306534,5,6,2,1,6,1,0,1,5,1,1,12086385,3,7,1,0,1,1,1,0,0,16,0,1,1,1,1
Credit card,226480,0,1,0,0,0,0,1,0,0,0,0,14567035,2,0,0,1,0,0,0,1,0,16,1,0,0,0,0
Dispute,4445,0,0,0,0,0,0,0,0,0,0,0,59520,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
No charge,8644,0,0,0,0,0,0,0,0,0,0,0,123566,0,0,0,0,0,0,0,0,1,9,0,0,0,0,0
Unknown,529772,0,0,0,0,0,0,0,0,0,0,0,413593,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [116]:
# Checking if there's any link between mta_tax and Ratecode using pivot table
pd.pivot_table(taxi_trips, index='total_amount_match?', columns='mta_tax', values='lpep_pickup_datetime', aggfunc='count', fill_value=0)

mta_tax,0.00,0.07,0.08,0.09,0.10,0.11,0.12,0.13,0.18,0.25,0.32,0.36,0.50,0.55,0.60,0.65,0.81,0.83,1.00,1.37,1.50,2.75,3.55,3.80,4.41,17.33,26.22,82.91
total_amount_match?,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Match,1063518,5,7,2,1,6,1,1,1,5,1,1,26671973,5,7,1,1,1,1,1,1,1,1,1,1,1,0,1
No Match,12357,0,0,0,0,0,0,0,0,0,0,0,578126,0,0,0,0,0,0,0,0,0,41,0,0,0,1,0


From the above pivot table, we can see that the total_amount for 42 records where the mta_tax isn't 0.00 or 0.50, doesn't match with the total_amount_check. Let's check those 42 records first.

In [117]:
# Checking the rows where the mta_tax is not 0.50 or 0.00 and where the total_amount_match? is No Match
taxi_trips.loc[~(taxi_trips['mta_tax']==0.50) & ~(taxi_trips['mta_tax']==0.00) & (taxi_trips['total_amount_match?']=='No Match')]

,VendorID,lpep_pickup_datetime,lpep_dropoff_datetime,store_and_fwd_flag,RatecodeID,PULocationID,DOLocationID,passenger_count,trip_distance,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,payment_type,trip_type,congestion_surcharge,Vendor,Ratecode,PU_Borough,PU_Zone,PU_service_zone,DO_Borough,DO_Zone,DO_service_zone,total_amount_check,total_amount_match?,total_amount_difference,trip_duration_mins
12271127,1,2018-04-28 22:25:49,2018-04-28 22:30:30,N,1,82,129,1,1.3,3005.0,25.42,26.22,0.00,0.00,0.0,3005.80,Cash,Street-hail,0.00,"Creative Mobile Technologies, LLC",Standard rate,Queens,Elmhurst,Boro Zone,Queens,Jackson Heights,Boro Zone,3056.64,No Match,-50.84,4.683333
21164280,1,2019-04-29 18:25:53,2019-04-29 18:25:55,N,1,230,230,1,8.2,30.0,3.75,3.55,0.00,0.00,0.0,37.30,Cash,Street-hail,2.75,"Creative Mobile Technologies, LLC",Standard rate,Manhattan,Times Sq/Theatre District,Yellow Zone,Manhattan,Times Sq/Theatre District,Yellow Zone,40.05,No Match,-2.75,0.033333
21164282,1,2019-05-19 19:29:21,2019-05-19 19:29:23,N,1,162,162,1,5.6,21.0,2.75,3.55,0.00,0.00,0.0,27.30,No charge,Street-hail,2.75,"Creative Mobile Technologies, LLC",Standard rate,Manhattan,Midtown East,Yellow Zone,Manhattan,Midtown East,Yellow Zone,30.05,No Match,-2.75,0.033333
21164284,1,2019-05-21 00:09:20,2019-05-21 00:09:23,N,1,107,107,1,8.5,36.5,3.25,3.55,0.00,0.00,0.0,43.30,Cash,Street-hail,2.75,"Creative Mobile Technologies, LLC",Standard rate,Manhattan,Gramercy,Yellow Zone,Manhattan,Gramercy,Yellow Zone,46.05,No Match,-2.75,0.050000
21164285,1,2019-05-25 16:32:38,2019-05-25 16:32:43,N,1,186,186,1,7.0,28.0,2.75,3.55,0.00,0.00,0.0,34.30,No charge,Street-hail,2.75,"Creative Mobile Technologies, LLC",Standard rate,Manhattan,Penn Station/Madison Sq West,Yellow Zone,Manhattan,Penn Station/Madison Sq West,Yellow Zone,37.05,No Match,-2.75,0.083333
21164286,1,2019-05-25 16:34:14,2019-05-25 16:34:20,N,1,186,186,1,7.1,28.5,2.75,3.55,0.00,0.00,0.0,34.80,No charge,Street-hail,2.75,"Creative Mobile Technologies, LLC",Standard rate,Manhattan,Penn Station/Madison Sq West,Yellow Zone,Manhattan,Penn Station/Madison Sq West,Yellow Zone,37.55,No Match,-2.75,0.100000
21164287,1,2019-05-25 18:12:20,2019-05-25 18:12:26,N,1,144,144,1,3.2,15.0,2.75,3.55,0.00,0.00,0.0,21.30,No charge,Street-hail,2.75,"Creative Mobile Technologies, LLC",Standard rate,Manhattan,Little Italy/NoLiTa,Yellow Zone,Manhattan,Little Italy/NoLiTa,Yellow Zone,24.05,No Match,-2.75,0.100000
21164288,1,2019-05-25 18:59:49,2019-05-25 18:59:53,N,1,161,161,1,6.4,21.5,2.75,3.55,0.00,0.00,0.0,27.80,No charge,Street-hail,2.75,"Creative Mobile Technologies, LLC",Standard rate,Manhattan,Midtown Center,Yellow Zone,Manhattan,Midtown Center,Yellow Zone,30.55,No Match,-2.75,0.066667
21164289,1,2019-05-27 17:26:56,2019-05-27 17:26:59,N,1,232,232,1,3.9,13.5,2.75,3.55,0.00,0.00,0.0,19.80,No charge,Street-hail,2.75,"Creative Mobile Technologies, LLC",Standard rate,Manhattan,Two Bridges/Seward Park,Yellow Zone,Manhattan,Two Bridges/Seward Park,Yellow Zone,22.55,No Match,-2.75,0.050000
21164290,1,2019-06-11 19:25:24,2019-06-11 19:25:30,N,1,261,261,1,2.2,9.5,3.75,3.55,0.00,0.00,0.0,16.80,No charge,Street-hail,2.75,"Creative Mobile Technologies, LLC",Standard rate,Manhattan,World Trade Center,Yellow Zone,Manhattan,World Trade Center,Yellow Zone,19.55,No Match,-2.75,0.100000


In [118]:
# Checking the total_amount_difference values where the mta_tax is not 0.50 or 0.00 and where the total_amount_match? is No Match
taxi_trips.loc[~(taxi_trips['mta_tax']==0.50) & ~(taxi_trips['mta_tax']==0.00) & (taxi_trips['total_amount_match?']=='No Match')].round(2).groupby(['mta_tax','total_amount_difference']).count()['VendorID']

mta_tax  total_amount_difference
3.55     -2.75                      41
26.22    -50.84                      1
Name: VendorID, dtype: int64

total_amount for 41 of those records were 2.75 less than total_amount_check. All 41 of those records were charged 2.75 as congestion_surcharge. Removing the congestion_surcharge of 2.75 would help match the total_amount with total_amount_check but this wouldn't be correct because mta_tax should never be values other than 0.00 or 0.50.

So, what would be the total_amount_difference if we are to update the 3.55 as 0.50? <br>
37.30 - (40.05 - 3.55 + 0.50) = 0.30

0.30 would be the total_amount_difference for the first record if we are to update the 3.55 as 0.50. Let's check if the total_amount_difference is 0.30 for the other 40 records.

In [119]:
# Assigning the 41 rows to a new variable to check what the total_amount would be after updating the values
mta_tax_check = taxi_trips.loc[~(taxi_trips['mta_tax']==0.50) & ~(taxi_trips['mta_tax']==0.00) & (taxi_trips['total_amount_difference'].round(2)==-2.75), ['fare_amount','extra','mta_tax','tip_amount','tolls_amount','improvement_surcharge','congestion_surcharge','total_amount','total_amount_check','total_amount_difference']]

mta_tax_check['total_amount_difference_after_update'] = mta_tax_check['total_amount'] - (mta_tax_check['total_amount_check'] - mta_tax_check['mta_tax'] + 0.50)

mta_tax_check['total_amount_difference_after_update'].round(1).value_counts()

0.3    41
Name: total_amount_difference_after_update, dtype: int64

For all 41 records, the total_amount_difference if we are to update the 3.55 as 0.50 would be 0.30. Therefore, we can update the mta_tax as 0.50 and improvement_surcharge as 0.30 where improvement_surcharge is 0.

In [120]:
# Getting the index numbers of rows to update
improvement_surcharge_to_update = taxi_trips.loc[(taxi_trips['mta_tax']==3.55) & (taxi_trips['total_amount_difference'].round(2)==-2.75) & (taxi_trips['improvement_surcharge']==0.0), 'improvement_surcharge'].index.to_list()

# Updating mta_tax as 0.5 using the index numbers
taxi_trips.loc[improvement_surcharge_to_update, 'mta_tax'] = 0.5

# Updating improvement_surcharge as 0.3 using the index numbers
taxi_trips.loc[improvement_surcharge_to_update, 'improvement_surcharge'] = 0.3

In [121]:
# Checking the rows again where the mta_tax is not 0.50 or 0.00 and where the total_amount_match? is No Match
taxi_trips.loc[~(taxi_trips['mta_tax']==0.50) & ~(taxi_trips['mta_tax']==0.00) & (taxi_trips['total_amount_match?']=='No Match')]

,VendorID,lpep_pickup_datetime,lpep_dropoff_datetime,store_and_fwd_flag,RatecodeID,PULocationID,DOLocationID,passenger_count,trip_distance,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,payment_type,trip_type,congestion_surcharge,Vendor,Ratecode,PU_Borough,PU_Zone,PU_service_zone,DO_Borough,DO_Zone,DO_service_zone,total_amount_check,total_amount_match?,total_amount_difference,trip_duration_mins
12271127,1,2018-04-28 22:25:49,2018-04-28 22:30:30,N,1,82,129,1,1.3,3005.0,25.42,26.22,0.0,0.0,0.0,3005.8,Cash,Street-hail,0.0,"Creative Mobile Technologies, LLC",Standard rate,Queens,Elmhurst,Boro Zone,Queens,Jackson Heights,Boro Zone,3056.64,No Match,-50.84,4.683333


Now, we are left with one record where the mta_tax is not 0.50 or 0.00 and where the total_amount_match? is No Match. The fare_amount for this row needs investigation as it isn't possible for a trip with the distance of 1.3 miles to have fare_amount of 3005. We will invetigate this later. But for now, we will update the mta_tax to be either 0 or 0.50.

In [122]:
# Checking if there's any link between mta_tax and Ratecode using pivot table
pd.pivot_table(taxi_trips.loc[taxi_trips['Ratecode']=='Standard rate'], index='Ratecode', columns='mta_tax', values='lpep_pickup_datetime', aggfunc='count', fill_value=0)

mta_tax,0.00,0.07,0.08,0.09,0.10,0.11,0.12,0.13,0.18,0.25,0.32,0.36,0.50,0.55,0.65,0.81,0.83,1.00,1.37,2.75,3.55,3.80,4.41,17.33,26.22,82.91
Ratecode,,,,,,,,,,,,,,,,,,,,,,,,,,
Standard rate,49870,5,6,2,1,6,1,1,1,5,1,1,26505429,5,1,1,1,1,1,1,1,1,1,1,1,1


MTA tax is automatically triggered based on the metered rate in use and from the above pivot table, we can see 99.8% of standard rate trips had 0.50 MTA tax. So we will update the 26.22 as 0.50 rather than 0.

In [123]:
# Checking the rows again where the mta_tax is not 0.50 or 0.00 and where the total_amount_match? is No Match
taxi_trips.loc[(taxi_trips['mta_tax']==26.22) & (taxi_trips['total_amount_match?']=='No Match'), 'mta_tax'] = 0.5

In [124]:
# Checking the total_amount_difference values where the mta_tax is not 0.50 or 0.00 and where the total_amount_match? is No Match
taxi_trips.loc[~(taxi_trips['mta_tax']==0.50) & ~(taxi_trips['mta_tax']==0.00)].round(2).groupby('total_amount_match?').count()['VendorID']

total_amount_match?
Match    55
Name: VendorID, dtype: int64

All unrecognised values of mta_tax in No Match category has been updated. Now, let's move on to the 55 rows where the total_amount_match? is Match.

---

To be continued...